In [2]:
!pip install spotipy

  Running setup.py bdist_wheel for spotipy ... - done
  Stored in directory: /root/.cache/pip/wheels/76/28/19/a86ca9bb0e32dbd4a4f580870250f5aeef852870578e0427e6
Successfully built spotipy


In [0]:
import pandas as pd
import nltk
import spotipy
import spotipy.util as util
import json
import numpy as np

In [0]:
key_file = 'keys.json'
keys = json.load(open(key_file))

SPOTIPY_CLIENT_ID= keys["client_id"]
SPOTIPY_CLIENT_SECRET= keys["client_secret"]
SPOTIPY_REDIRECT_URI= keys["redirect_url"]

In [9]:
token = util.prompt_for_user_token("1274418332", 
                                   scope = "user-library-read", 
                                   client_id=SPOTIPY_CLIENT_ID, 
                                   client_secret=SPOTIPY_CLIENT_SECRET, 
                                   redirect_uri = SPOTIPY_REDIRECT_URI)
spotify = spotipy.Spotify(auth=token)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=b29ebdb5e8ac4efaa4e1afe0da5a7e7c&response_type=code&redirect_uri=http%3A%2F%2Fgoogle.com.mx%2F&scope=user-library-read in your browser


Enter the URL you were redirected to: https://www.google.com.mx/?code=AQD2KxpZpk3EpeCSAKRc1edeH79WNC8ZMR7kXnsTWvhycleYFHoQITJkSSWjXDf_AtC2LJh1a9Gl9SA-3hFbblMnT8aISisZ3rqtWLymrw5gRjvV_YF-kWClQJ-u50f__oAGsW_vOiWtF6GkJidKX6t_484Q6UfFAJ6rbMpyX2wLzCEzFh5WL0bfzqtFjJXtm3-dGz3vG5v9a06gbyTHtw




In [11]:
billboard = pd.read_csv("billboard_lyrics_1964-2015.csv")
billboard.head()

,Rank,Song,Artist,Year,Lyrics,Source
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3.0
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love yo...,1.0
2,3,i cant get no satisfaction,the rolling stones,1965,,1.0
3,4,you were on my mind,we five,1965,when i woke up this morning you were on my mi...,1.0
4,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss...,1.0


In [12]:
temp = billboard.sort_values('Year',ascending=False)
temp.head()

,Rank,Song,Artist,Year,Lyrics,Source
5099,100,how deep is your love,calvin harris and disciples,2015,i want you to breathe me in let me be your ai...,1.0
5036,37,talking body,tove lo,2015,bed stay in bed the feeling of your skin lock...,1.0
5026,27,good for you,selena gomez featuring aap rocky,2015,im on my 14 carats im 14 carat doing it up li...,1.0
5027,28,all about that bass,meghan trainor,2015,because you know im all about that bass bout ...,1.0
5028,29,style,taylor swift,2015,midnight you come and pick me up no headlight...,1.0


In [16]:
calvin = spotify.search(q='artist:' + 'calvin harris and disciples' + ' track:' + 'how deep is your love', type='track')
calvin_uri = calvin['tracks']['items'][0]['uri']
calvin_id = calvin_uri.split('spotify:track:')[1]
calvin_id

'6rMwYjoFvxKiaEH4If8ZpZ'

In [17]:
calvin_features = spotify.audio_features('6rMwYjoFvxKiaEH4If8ZpZ')
calvin_features[0]['valence']

0.111

In [19]:
wooly = spotify.search(q='artist:' + 'sam the sham and the pharaohs' + ' track:' + 'wooly bully', type='track')
wooly_uri = wooly['tracks']['items'][0]['uri']
wooly_id = wooly_uri.split('spotify:track:')[1]
wooly_id

'2XkuSbp5say8nZW8g6156Z'

In [20]:
wooly_features = spotify.audio_features('6rMwYjoFvxKiaEH4If8ZpZ')
wooly_features[0]['valence']

0.111

In [0]:
def get_valence(song, artist, year):
    try:
        song = spotify.search(q='artist:' + artist + ' track:' + song, type='track')
        song_uri = song['tracks']['items'][0]['uri']
        song_id = song_uri.split('spotify:track:')[1]
        song_features = spotify.audio_features(song_id)
        valence = song_features[0]['valence']
        print('success: ' + str(year))
        return valence
    except IndexError as error:
        print('fail')
        return np.nan

In [57]:
billboard.head()

,Rank,Song,Artist,Year,Lyrics,Source
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3.0
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love yo...,1.0
2,3,i cant get no satisfaction,the rolling stones,1965,,1.0
3,4,you were on my mind,we five,1965,when i woke up this morning you were on my mi...,1.0
4,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss...,1.0


In [58]:
billboard['Valence'] = billboard.apply(lambda row: get_valence(row['Song'], row['Artist'], row['Year']), axis=1)

success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
fail
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
fail
fail
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
retrying ...1secs
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
success: 1965
s

KeyboardInterrupt: ignored